# Estimation of parameters of various magnetic cores

This notebook helps to estimate basic parameters of various cores salvaged from some broken equipment.
The math is mostly based on this [Guide for calculation of the effective parameters of magnetic piece parts](static/papers/effective-parameters-of-magnetic-parts.pdf) publised by Indian Standards Institution. Once calculated, the 'Core' object can then be added to [edcalc/components.py](edcalc/components.py) for future use or copied
directly into a respective notebook.

## $ A_L $ value

$A_L$, or inductance per square turn (measured in $nH / N^2$) is easy to measure and it can later
be used for practical calculations and derivation of some other parameters.

Knowing $ A_L $, the inductance can then be found as $ L = A_L N^2 $, where N is the number of turns in the winding. Also, $$A_L = {\mu \mu_0 A_e \over l_e}$$ where 
* $\mu$ is the relative permeability (unitless, depends on the core material)
* $\mu_0$ is the permeability of free space (a constant equal to $4 \pi \times 10^{−7}$ H/m),
* $A_e$ is the effective cross-sectional area of the core ($meters^2$)
* $l_e$ is the effective magnetic path ($meters$)

Both $A_e$ and $l_e$ can be found after measuring the geometry of the core (see the sections below).
$ A_L $ can be found by winding some wire on the core and then measuring the inductance. This allows us to find
relative permeability as $$\mu = {A_L l_e \over A_e \mu_0}$$

In [18]:
# Necessary python imports
import math
from IPython.display import display, Markdown

# Here is a short code snippet for calculating A_L based
# on several measurements of the inductance made with
# different numbers of turns.

# (<the number of turns>, <measured inductance in nH>)
L_data_points = [
    (10, 10000),
    (18, 34000),
]
A_L_points = [(L / N**2) for N, L in L_data_points]
display(Markdown(f'$A_L$ data points: {[round(p, 2) for p in A_L_points]}'))
A_L_average = sum(A_L_points) / len(A_L_points)
display((Markdown(f'Average $A_L$ = {round(A_L_average, 2)} $nH / N^2$')))

$A_L$ data points: [100.0, 104.94]

Average $A_L$ = 102.47 $nH / N^2$

The rest will depend on what type of core you have

## Toroid core

In [20]:
import math
from edcalc.magnetics import Core

def toroid_core_params(outer_dia, inner_dia, height, corner_radius=0, A_L=None) -> Core:
    """
    Calculates effective parameters of a toroid core based on its geometric shape.
    :param A_L: you can specify A_L if you know it. It will be added to the resulting structure.
    """
    r1, r2 = inner_dia / 2, outer_dia / 2
    k1 = 0.8584 * corner_radius**2 / (height * (r2 - r1))
    he = height * (1 - k1)
    c1 = 2 * math.pi / (he * math.log(r2 / r1))
    c2 = 2 * math.pi * (1 / r1 - 1 / r2) / (he**2 * math.log(r2 / r1)**3)
    return Core(A_e=c1 / c2, V_e=c1**3 / c2**2, l_e=c1**2 / c2, A_L=A_L)

print(toroid_core_params(35.7, 25.15, 2.62, A_L=100))

Core(mu=544.8005179787447, l_e=93.6558305883444, A_e=13.680042415170472, V_e=1281.2157348765716, A_L=100, B_sat=0.28, W_a=None, note='')


## Pot core

![Pot core geometry](static/images/pot-core-geometry.gif)

In [1]:
import math
from edcalc.magnetics import Core

def pot_core_params(d1, d2, d3, d4, h1, h2, n_slots=0, slot_area=0, A_L=None) -> Core:
    """
    All parameters should be obvious from the picture above and should be given in millimeters.
    Except it doesn't use parameter 'c', because some pot cores
    have one or two side slots of various length instead.
    So 'c' was replaced with the 'slot_area'.
    :param slot_area: is the area of the hole through the side, and its area
        is given as if looking from the side of the core.
    :param n_slots: the number of slots we have
    :param A_L: you can specify A_L if you know it. It will be added to the resulting structure.
    """
    r1, r2, r3, r4 = d4/2, d3/2, d2/2, d1/2    
    s1 = r2 - math.sqrt((r1**2 + r2**2) / 2)
    s2 = - r3 + math.sqrt((r3**2 + r4**2) / 2)
    A = [0, 0, 0, 0, 0]
    l1 = h1 + h2
    l2 = d2 / 2 - d3 / 2
    l3 = 2 * h2
    h = (h1 - h2)
    l4 = (math.pi / 4) * (2 * s2 + h)
    l5 = (math.pi / 4) * (2 * s1 + h)
    l2_A2 = (1 / (math.pi * h)) * math.log(r3 / r2)
    l2_A2s = (1 / (2 * math.pi**2 * h**2)) * (r3 - r2)/(r3 * r2)
    A1 = math.pi * (d1/2 - d2/2)*(d1/2 + d2/2)
    A3 = math.pi * (d3/2 - d4/2)*(d3/2 + d4/2)
    A4 = (math.pi/2) * (r4**2 - r3**2 + 2*r3*h)
    A5 = (math.pi/2) * (r2**2 - r1**2 + 2*r2*h)
    C1 = l1/A1 + l2_A2 + l3 / A3 + l4 / A4 + l5 / A5
    C2 = l1/A1**2 + l2_A2s + l3 / A3**2 + l4 / A4**2 + l5 / A5**2
    Ae = C1 / C2
    le = C1**2 / C2
    Ve = C1**3 / C2**2 - n_slots * slot_area * h
    Ae = Ve / le
    return Core(A_e=Ae, V_e=Ve, l_e=le, A_L=A_L)
    
# Example:
print(pot_core_params(21, 18, 8.7, 4.4, 6.85, 4.8, 2, 12.4, A_L=100))

Core(mu=44.5000561785827, l_e=32.26492549023729, A_e=57.69793143243745, V_e=1861.6194586084143, A_L=100, B_sat=0.28, W_a=None, note='')
